In [1]:
from typing import Union
from tqdm import tqdm

import pandas as pd
import numpy as np
import plotly.express as px

from scipy import stats
from statsmodels.stats.meta_analysis import effectsize_smd
from statsmodels.stats import proportion
from statsmodels.stats.power import tt_ind_solve_power
from statsmodels.stats.power import zt_ind_solve_power

In [2]:
data = pd.read_csv('D:\\Nastya\\GeekBrains\\GeekBrains\\АВ тестирование\\Занятие 9\\gb_sem_9_hw.csv')

In [3]:
data
# gate_30 - контроль, 40 - тест
# нежно сделать три проверки: 1) по метрике sum_gamerounds, 2) по метрике retention_1, 3) по метрике retention_7
# True и False через .sum  суммируются (не обяз-но переводить в 0 и 1); проверить чтобы читались как булевые, а не как строки

,userid,version,sum_gamerounds,retention_1,retention_7
0,116,gate_30,3,False,False
1,337,gate_30,38,True,False
2,377,gate_40,165,True,False
3,483,gate_40,1,False,False
4,488,gate_40,179,True,True
...,...,...,...,...,...
90184,9999441,gate_40,97,True,False
90185,9999479,gate_40,30,False,False
90186,9999710,gate_30,28,True,False
90187,9999768,gate_40,51,True,False


In [4]:
# булевые метрики могут складываться, чтобы получить вероятность
pd.read_csv('D:\\Nastya\\GeekBrains\\GeekBrains\\АВ тестирование\\Занятие 9\\gb_sem_9_hw.csv').retention_1.sum()

40153

In [5]:
data.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 90189 entries, 0 to 90188
Data columns (total 5 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   userid          90189 non-null  int64 
 1   version         90189 non-null  object
 2   sum_gamerounds  90189 non-null  int64 
 3   retention_1     90189 non-null  bool  
 4   retention_7     90189 non-null  bool  
dtypes: bool(2), int64(2), object(1)
memory usage: 2.2+ MB


In [6]:
data.shape

(90189, 5)

In [7]:
data.describe()

,userid,sum_gamerounds
count,9.018900e+04,90189.000000
mean,4.998412e+06,51.872457
std,2.883286e+06,195.050858
min,1.160000e+02,0.000000
25%,2.512230e+06,5.000000
50%,4.995815e+06,16.000000
75%,7.496452e+06,51.000000
max,9.999861e+06,49854.000000


In [33]:
control.shape, treatment.shape

Группы различаются на 789 позиций. Это не существенная разница. 

((44700, 5), (45489, 5))

In [8]:
control = data[data['version'] == 'gate_30'].copy(deep=True) 
treatment = data[data['version'] == 'gate_40'].copy(deep=True)

In [9]:
control.describe()

,userid,sum_gamerounds
count,4.470000e+04,44700.000000
mean,4.987564e+06,52.456264
std,2.881026e+06,256.716423
min,1.160000e+02,0.000000
25%,2.505469e+06,5.000000
50%,4.983631e+06,17.000000
75%,7.481497e+06,50.000000
max,9.999710e+06,49854.000000


In [10]:
treatment.describe()

,userid,sum_gamerounds
count,4.548900e+04,45489.000000
mean,5.009073e+06,51.298776
std,2.885496e+06,103.294416
min,3.770000e+02,0.000000
25%,2.517171e+06,5.000000
50%,5.007329e+06,16.000000
75%,7.510762e+06,52.000000
max,9.999861e+06,2640.000000


### Проверка 1 по метрике sum_gamerounds

In [11]:
def continious_result(control: pd.DataFrame,
                      treatment: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Статистика по выборкам (вытащить описательные статистики: размер выборки, среднее контрольной и тестовой групп, станд откл обеих групп)
    size = control.loc[:, column].shape[0]
    
    control_mean = control.loc[:, column].mean()
    treatment_mean = treatment.loc[:, column].mean()
    
    control_std = control.loc[:, column].std(ddof=1)
    treatment_std = treatment.loc[:, column].std(ddof=1)
    
#  теперь нам нужно посчитать либо при нормальном распределении  Т-критерий, либо при ненормальном - критерий манна уитни
#  ЛИБО пойти другим путем и посчитать через БУТСТРАП
# Преимущества БУТСТРАПА: 
# 1 - с него мы сразу получаем распределение разницы наших средних
# 2 - на его основе можно посчитать через генератор распределение значений p-value
# 3 - если был бы А/А тест, могли бы посчитать значение ошибки 1 рода разниц между нашими группами, будет она или нет.

    # Бутсрап
    booted_diff = [] # создаем лист, куда мы будем записывать БУТСТРАПированную разницу
    for _ in tqdm(range(n_iters)): # запускаем цикл по количеству итераций
        control_sample = control.loc[:, column].sample(n=size, replace=True).values #семплируем контроль (нужно выделить колонку, которую мы семплируем, и передать ему размер выборки, кот мы семплируем)
        treatment_sample = treatment.loc[:, column].sample(n=size, replace=True).values # replace=True - чтобы были замены и возвраты, чтобы не было всегда одного и того же результата
        booted_diff.append(np.mean(control_sample - treatment_sample)) # считаем разницу между контролем и тестом и записываем среднее значение этой разницы в массив booted_diff
# т.о. мы получим распределение бутстрапированный разницы средних наших выборок
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1) # md_ci - средрнее значение (т.е. средний эффект межд значениями в группах), std_ci - стд отклонение этого эффекта 
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5]) # левая и правая границы доверительного интервала
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci))) # считаем значение p-value через генератора норм распр-ия и суммы куммулятивной ф-ии
    
    # Считаем мощность эксперимента
    effect_size, _ = effectsize_smd(mean1=treatment_mean, sd1=treatment_std, nobs1=size,
                                    mean2=control_mean, sd2=control_std, nobs2=size)
    power = tt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, # ошибка 1 рода
                           'beta': (1-power), # ошибка 2 рода
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',  # доверит интвервал (лев и прав границы округлим до 3 знаков после запятой)
                           'difference': md_ci,}, # разница между нашими группами
                          index=[column]) 
    return result

In [12]:
continious_result(control, treatment, column='sum_gamerounds')

100%|██████████| 10000/10000 [00:45<00:00, 218.12it/s]


,effect_size,alpha,beta,CI,difference
sum_gamerounds,-0.005915,0.376575,0.856725,"[-0.978, 4.095]",1.156831


In [ ]:
# Вывод: разница между группами по данной метрике статистически не значима.
# alpha = 0,376 - это p-value, намного больше чем 5%
# Высокая ошибка 2 рода, т.к. beta = 86%, а должно быть в норме ~ 20%, т.е. 0,02
# Доверительный интервал [-0.978, 4.095] включает в себя 0. Есть выбросы справа?!

### Проверка 2 по метрике retention_1.

In [34]:
def proportion_result(control: pd.DataFrame,
                      treatment: pd.DataFrame,
                      column: str,
                      n_iters: int = 10_000) -> pd.DataFrame:
    # Вероятность событий (в предыдущем считали статистику, а темепрь вероятность событий)
    size = control.loc[:, column].shape[0]
    prop_control = control.loc[:, column].sum() / size # отношение количество сковенртированных пользователей к общему количеству
    prop_treatment = treatment.loc[:, column].sum() / size
    
    # Бутсрап
    booted_diff = []
    for _ in tqdm(range(n_iters)):
        control_sample = stats.bernoulli.rvs(p=prop_control, size=size) # не семплируем, а делаем новую генерацию распределения бернулли с той же вероятностью, которая возможна
        treatment_sample = stats.bernoulli.rvs(p=prop_treatment, size=size)
        booted_diff.append(np.mean(control_sample - treatment_sample))
    
    # Считаем статистику после бустрапа
    md_ci, std_ci = np.mean(booted_diff), np.std(booted_diff, ddof=1)
    left_ci, right_ci = np.percentile(booted_diff, [2.5, 97.5])
    p_value_ci = 2 * (1 - stats.norm.cdf(np.abs(md_ci / std_ci)))
    
    # Считаем мощность эксперимента
    effect_size = proportion.proportion_effectsize(prop_control, prop_treatment)
    
    power = zt_ind_solve_power(effect_size=effect_size,
                               nobs1=size,
                               alpha=.05,
                               power=None,
                               ratio=1)
    # Формируем отчёт 
    result = pd.DataFrame({'effect_size': effect_size,
                           'alpha': p_value_ci, 
                           'beta': (1-power), 
                           'CI': f'[{np.round(left_ci, 3)}, {np.round(right_ci, 3)}]',
                           'difference': md_ci,}, 
                          index=[column]) 
    return result

In [35]:
proportion_result(control, treatment, column='retention_1')

100%|██████████| 10000/10000 [01:29<00:00, 112.30it/s]


,effect_size,alpha,beta,CI,difference
retention_1,-0.003823,0.571713,0.911819,"[-0.008, 0.005]",-0.00189


In [ ]:
# Вывод: разница между группами по данной метрике статистически не значима.
# alpha = %57 - это p-value, намного больше чем 5%
# Высокая ошибка 2 рода, т.к. beta = 91%, а должно быть в норме ~ 20%, т.е. 0,02
# Доверительный интервал [-0.008, 0.005] включает в себя 0.
# Difference разница между группами очень маленькая.

### Проверка 2 по метрике retention_7.

In [36]:
proportion_result(control, treatment, column='retention_7')

100%|██████████| 10000/10000 [00:45<00:00, 221.24it/s]


,effect_size,alpha,beta,CI,difference
retention_7,0.012776,0.057073,0.519844,"[-0.0, 0.01]",0.004965


In [ ]:
# Вывод: разница между группами по данной метрике статистически не значима.
# alpha = 0,057 - это p-value, совсем чуть-чуть больше чем 5%
# Высокая ошибка 2 рода, т.к. beta = 52%, а должно быть в норме ~ 20%, т.е. 0,02
# Доверительный интервал [-0.0, 0.01] включает в себя 0.
# Difference разница между группами очень маленькая.
# Результаты вызывают неоднозначные выводы, но в целом можно сказать, что нет статистически значимых различий.